In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 

pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_tensor_type(torch.DoubleTensor)


In [2]:
def PiecewiseGQ3D(target, Nx, order): 
    """
    Parameters
    ----------
    target : 
        Target function 
    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature
    """

    # print("order: ",order )
    x, w = np.polynomial.legendre.leggauss(order)
    gauss_pts = np.array(np.meshgrid(x,x,x,indexing='ij')).reshape(3,-1).T
    weight_list = np.array(np.meshgrid(w,w,w,indexing='ij'))
    weights =   (weight_list[0]*weight_list[1]*weight_list[2]).ravel() 

    gauss_pts =torch.tensor(gauss_pts)
    weights = torch.tensor(weights)

    h = 1/Nx # 100 intervals 
    long_weights =  torch.tile(weights,(Nx**3,1))
    long_weights = long_weights.reshape(-1,1)
    long_weights = long_weights * h**3 /8 

    integration_points = torch.tile(gauss_pts,(Nx**3,1))
    scale_factor = h/2 
    integration_points = scale_factor * integration_points

    index = np.arange(1,Nx+1)-0.5
    ordered_pairs = np.array(np.meshgrid(index,index,index,indexing='ij'))
    ordered_pairs = ordered_pairs.reshape(3,-1).T

    # print(ordered_pairs)
    # print()
    ordered_pairs = torch.tensor(ordered_pairs)
    # print(ordered_pairs.size())
    ordered_pairs = torch.tile(ordered_pairs, (1,order**3)) # number of GQ points
    # print(ordered_pairs)

    ordered_pairs =  ordered_pairs.reshape(-1,3)
    # print(ordered_pairs)
    translation = ordered_pairs*h 
    # print(translation)

    integration_points = integration_points + translation 
    integration_points = integration_points.to(device)
    # print(integration_points.size())
    # func_values = integrand2_torch(integration_points)
    func_values = target(integration_points)

    long_weights = long_weights.to(device) 
    integral_value = long_weights.T@func_values # this allows multiple functions, stored in columns
    return integral_value


def PiecewiseGQ3D_weights_points(Nx, order): 
    """ A slight modification of PiecewiseGQ2D function that only needs the weights and integration points.
    Parameters
    ----------

    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature

    Returns
    -------
    long_weights: torch.tensor
    integration_points: torch.tensor
    """

    """
    Parameters
    ----------
    target : 
        Target function 
    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature
    """

    # print("order: ",order )
    x, w = np.polynomial.legendre.leggauss(order)
    gauss_pts = np.array(np.meshgrid(x,x,x,indexing='ij')).reshape(3,-1).T
    weight_list = np.array(np.meshgrid(w,w,w,indexing='ij'))
    weights =   (weight_list[0]*weight_list[1]*weight_list[2]).ravel() 

    gauss_pts =torch.tensor(gauss_pts)
    weights = torch.tensor(weights)

    h = 1/Nx # 100 intervals 
    long_weights =  torch.tile(weights,(Nx**3,1))
    long_weights = long_weights.reshape(-1,1)
    long_weights = long_weights * h**3 /8 

    integration_points = torch.tile(gauss_pts,(Nx**3,1))
    # print("shape of integration_points", integration_points.size())
    scale_factor = h/2 
    integration_points = scale_factor * integration_points

    index = np.arange(1,Nx+1)-0.5
    ordered_pairs = np.array(np.meshgrid(index,index,index,indexing='ij'))
    ordered_pairs = ordered_pairs.reshape(3,-1).T

    # print(ordered_pairs)
    # print()
    ordered_pairs = torch.tensor(ordered_pairs)
    # print(ordered_pairs.size())
    ordered_pairs = torch.tile(ordered_pairs, (1,order**3)) # number of GQ points
    # print(ordered_pairs)

    ordered_pairs =  ordered_pairs.reshape(-1,3)
    # print(ordered_pairs)
    translation = ordered_pairs*h 
    # print(translation)

    integration_points = integration_points + translation 

    return long_weights, integration_points



def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct"):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    jac = weighted_basis_value_col.t() @ basis_value_col 
     
    rhs = weighted_basis_value_col.t() @ (target(integration_points)) 
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 


def minimize_linear_layer(model,target, solver="direct",Nx = 50, order =3):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    def loss_function_inside(x):
        #my_model is a global variable
        return 0.5*torch.pow(model(x)-target(x),2).to(device)

    def rhs_loss_inside(x): 
        # Helper function: differentiate this wrt linear layer parameters give the rhs
        return model(x)*target(x)

    loss = PiecewiseGQ3D(loss_function_inside, Nx,order)
    # Extract the linear system A :  
    du1 = torch.autograd.grad(outputs=loss, inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    jac = '' # Fix a bug: my_model -> model 2023 Mar. 14th 
    torch.set_printoptions(precision=8)
    neuron_number = model.fc1.bias.size(0)
    jac = torch.cat([torch.autograd.grad(outputs=du1[0,i], inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0] for i in range(neuron_number)],dim= 0 )

    # for i in range(neuron_number): # Fix a bug: hidden_size1 -> neuron_number 2023 Mar. 14th 
    #     duui = torch.autograd.grad(outputs=du1[0,i], inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    #     if i == 0: # Fix a bug: my_model -> model 2023 Mar. 14th 
    #         jac = duui
    #     else: 
    #         jac = torch.cat([jac,duui],dim=0)

    loss_helper = PiecewiseGQ3D(rhs_loss_inside, Nx, order)
    rhs = torch.autograd.grad(outputs=loss_helper, inputs=model.fc2.weight, retain_graph=True,create_graph = True)[0]
    rhs = rhs.view(-1,1)

    # Solve the linear system using different solvers 
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
#         sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    return sol 


In [3]:
class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1




## 3D QMC greedy algorithm 


In [4]:
def generate_relu_dict3D(N_list):
    N1 = N_list[0]
    N2 = N_list[1]
    N3 = N_list[2]
    
    N = N1*N2*N3 
    theta1 = np.linspace(0, pi, N1, endpoint= True).reshape(N1,1)
    theta2 = np.linspace(0, 2*pi, N2, endpoint= False).reshape(N2,1)
    b = np.linspace(-1.732, 1.732, N3,endpoint=False).reshape(N3,1) # threshold: 3**0.5  
    coord3 = np.array(np.meshgrid(theta1,theta2,b,indexing='ij'))
    coord3 = coord3.reshape(3,-1).T # N1*N2*N3 x 3. coordinates for the grid points 
    coord3 = torch.tensor(coord3) 

    f1 = torch.zeros(N,1) 
    f2 = torch.zeros(N,1)
    f3 = torch.zeros(N,1)
    f4 = torch.zeros(N,1)

    f1[:,0] = torch.cos(coord3[:,0]) 
    f2[:,0] = torch.sin(coord3[:,0]) * torch.cos(coord3[:,1])
    f3[:,0] = torch.sin(coord3[:,0]) * torch.sin(coord3[:,1])
    f4[:,0] = coord3[:,2] 

    Wb_tensor = torch.cat([f1,f2,f3,f4],1) # N x 4 
    return Wb_tensor


def generate_relu_dict3D_QMC(s,N0):
#     Sob = torch.quasirandom.SobolEngine(dimension =3, scramble= True, seed=None) 
#     samples = Sob.draw(N0).double() 

#     for i in range(s-1):
#         samples = torch.cat([samples,Sob.draw(N0).double()],0)

    # Monte Carlo 
    samples = torch.rand(s*N0,3) 
    T =torch.tensor([[pi,0,0],[0,2*pi,0],[0,0,1.732*2]])
    shift = torch.tensor([0,0,-1.732])
    samples = samples@T + shift 

    f1 = torch.zeros(s*N0,1) 
    f2 = torch.zeros(s*N0,1)
    f3 = torch.zeros(s*N0,1)
    f4 = torch.zeros(s*N0,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1])
    f4[:,0] = samples[:,2] 

    Wb_tensor = torch.cat([f1,f2,f3,f4],1) # N x 4 
    return Wb_tensor

def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights.to(device), integration_points.to(device) 


# ## Test 
# relu_dict = generate_relu_dict3D(2,2,2) # 1000 x 4
# print(relu_dict)

# s = 2 
# N0= 5 
# Wb = generate_relu_dict3D_QMC(s,N0)
# print(Wb.size())
# print(Wb)

In [5]:

def OGAL2FittingReLU3D(my_model,target,N_list,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "direct"): 

    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    gw_expand, integration_points = PiecewiseGQ3D_weights_points(Nx, order)
    gw_expand = gw_expand.to(device)
    integration_points = integration_points.to(device)

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.sum(func_values_sqrd*gw_expand)**0.5
    start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
#     relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
    N = np.prod(N_list) 
    relu_dict_parameters = generate_relu_dict3D(N_list).to(device)
    
    for i in range(num_epochs): 
#         relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        weight_func_values = func_values*gw_expand  
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:3].T ) - relu_dict_parameters[:,3])**k).T # uses broadcasting

        output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
        neuron_index = torch.argmax(output.flatten())
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:3]) # 
        list_b.append(-relu_dict_parameters[neuron_index,3])
        num_neuron += 1
        my_model = model(3,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

#         sol = minimize_linear_layer(my_model,target,solver,Nx,order)
        sol = minimize_linear_layer_explicit_assemble(my_model,target,gw_expand,integration_points,solver)

        my_model.fc2.weight.data[0,:] = sol[:]
        # if (i+1)%plot_freq == 0: 
        #     plot_2D(my_model.cpu())
        #     my_model = my_model.to(device)

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.sum(func_values_sqrd*gw_expand)**0.5
    print("time taken: ",time.time() - start_time)
    return err, my_model



def OGAL2FittingReLU3D_QMC(my_model,target,s,N0,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "direct"): 

    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    gw_expand, integration_points = PiecewiseGQ3D_weights_points(Nx, order)
    gw_expand = gw_expand.to(device)
    integration_points = integration_points.to(device)

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.sum(func_values_sqrd*gw_expand)**0.5
    start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        weight_func_values = func_values*gw_expand  
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:3].T ) - relu_dict_parameters[:,3])**k).T # uses broadcasting

        output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
        neuron_index = torch.argmax(output.flatten())
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:3]) # 
        list_b.append(-relu_dict_parameters[neuron_index,3])
        num_neuron += 1
        my_model = model(3,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]


#         sol = minimize_linear_layer(my_model,target,solver,Nx,order)
        sol = minimize_linear_layer_explicit_assemble(my_model,target,gw_expand,integration_points,solver)

        my_model.fc2.weight.data[0,:] = sol[:]
        # if (i+1)%plot_freq == 0: 
        #     plot_2D(my_model.cpu())
        #     my_model = my_model.to(device)

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.sum(func_values_sqrd*gw_expand)**0.5
    print("time taken: ",time.time() - start_time)
    return err, my_model



def OGAL2FittingReLU3D_QMC(my_model,target,s,N0,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "direct"): 

    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    gw_expand, integration_points = PiecewiseGQ3D_weights_points(Nx, order)
    gw_expand = gw_expand.to(device)
    integration_points = integration_points.to(device)

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    


    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.sum(func_values_sqrd*gw_expand)**0.5
    start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict3D_QMC(s,N0).to(device) 
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        weight_func_values = func_values*gw_expand  
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:3].T ) - relu_dict_parameters[:,3])**k).T # uses broadcasting

        output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
        neuron_index = torch.argmax(output.flatten())
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:3]) # 
        list_b.append(-relu_dict_parameters[neuron_index,3])
        num_neuron += 1
        my_model = model(3,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]


#         sol = minimize_linear_layer(my_model,target,solver,Nx,order)
        sol = minimize_linear_layer_explicit_assemble(my_model,target,gw_expand,integration_points,solver)

        my_model.fc2.weight.data[0,:] = sol[:]
        # if (i+1)%plot_freq == 0: 
        #     plot_2D(my_model.cpu())
        #     my_model = my_model.to(device)

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.sum(func_values_sqrd*gw_expand)**0.5
    print("time taken: ",time.time() - start_time)
    return err, my_model





## Randomized dictionaries

In [8]:
def target(x):
    return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) 

function_name = "sinpipipi" 
filename_write = "data/3DQMCOGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 

for N0 in [2*1,2**2,2**3,2**4,2**5]: 
    save = True  
    f_write = open(filename_write, "a")
    my_model = None 
    s = 2**5  
    Nx = 25   
    order = 3    
    exponent = 9  
    num_epochs = 2**exponent  
    plot_freq = 1 
    err_QMC2, my_model = OGAL2FittingReLU3D_QMC(my_model,target,s,N0,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "ls")
    
    if save: 
        folder = 'data/'
        filename = folder + 'err_OGA_3D_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,num_epochs,s*N0)
        torch.save(err_QMC2,filename) 
        folder = 'data/'
        filename = folder + 'model_OGA_3D_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,num_epochs,s*N0)
        torch.save(my_model,filename)

    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_QMC2[i] for i in neuron_nums ]
    f_write.write('dictionary size: {}\n'.format(s*N0))
    f_write.write("neuron num \t\t error \t\t order\n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
            f_write.write("{} \t\t {} \t\t * \n".format(neuron_nums[i],item))
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2)))
    f_write.write("\n")
    f_write.close()



using linear solver:  ls
epoch:  1	assembling the matrix time taken:  0.00024366378784179688
solving Ax = b time taken:  0.00026679039001464844
epoch:  2	assembling the matrix time taken:  0.00025200843811035156
solving Ax = b time taken:  0.0009253025054931641
epoch:  3	assembling the matrix time taken:  0.00024437904357910156
solving Ax = b time taken:  0.000843048095703125
epoch:  4	assembling the matrix time taken:  0.00026416778564453125
solving Ax = b time taken:  0.0008518695831298828
epoch:  5	assembling the matrix time taken:  0.0002446174621582031
solving Ax = b time taken:  0.0009307861328125
epoch:  6	assembling the matrix time taken:  0.00025343894958496094
solving Ax = b time taken:  0.0009648799896240234
epoch:  7	assembling the matrix time taken:  0.0002567768096923828
solving Ax = b time taken:  0.0010139942169189453
epoch:  8	assembling the matrix time taken:  0.0002532005310058594
solving Ax = b time taken:  0.0010585784912109375
epoch:  9	assembling the matrix time 

solving Ax = b time taken:  0.0062258243560791016
epoch:  76	assembling the matrix time taken:  0.00024318695068359375
solving Ax = b time taken:  0.006147146224975586
epoch:  77	assembling the matrix time taken:  0.0002415180206298828
solving Ax = b time taken:  0.00635981559753418
epoch:  78	assembling the matrix time taken:  0.0002410411834716797
solving Ax = b time taken:  0.006364583969116211
epoch:  79	assembling the matrix time taken:  0.0002415180206298828
solving Ax = b time taken:  0.006562232971191406
epoch:  80	assembling the matrix time taken:  0.0002608299255371094
solving Ax = b time taken:  0.006334781646728516
epoch:  81	assembling the matrix time taken:  0.000247955322265625
solving Ax = b time taken:  0.007003307342529297
epoch:  82	assembling the matrix time taken:  0.00024437904357910156
solving Ax = b time taken:  0.00691676139831543
epoch:  83	assembling the matrix time taken:  0.0002484321594238281
solving Ax = b time taken:  0.0071773529052734375
epoch:  84	ass

epoch:  150	assembling the matrix time taken:  0.0002551078796386719
solving Ax = b time taken:  0.013992786407470703
epoch:  151	assembling the matrix time taken:  0.00025153160095214844
solving Ax = b time taken:  0.01411890983581543
epoch:  152	assembling the matrix time taken:  0.00025582313537597656
solving Ax = b time taken:  0.01374197006225586
epoch:  153	assembling the matrix time taken:  0.00026416778564453125
solving Ax = b time taken:  0.014246463775634766
epoch:  154	assembling the matrix time taken:  0.0002548694610595703
solving Ax = b time taken:  0.014134407043457031
epoch:  155	assembling the matrix time taken:  0.0002536773681640625
solving Ax = b time taken:  0.014420270919799805
epoch:  156	assembling the matrix time taken:  0.00026035308837890625
solving Ax = b time taken:  0.014023780822753906
epoch:  157	assembling the matrix time taken:  0.00025200843811035156
solving Ax = b time taken:  0.014615297317504883
epoch:  158	assembling the matrix time taken:  0.0002

solving Ax = b time taken:  0.02262091636657715
epoch:  224	assembling the matrix time taken:  0.00024509429931640625
solving Ax = b time taken:  0.022095680236816406
epoch:  225	assembling the matrix time taken:  0.00024580955505371094
solving Ax = b time taken:  0.02279210090637207
epoch:  226	assembling the matrix time taken:  0.0002448558807373047
solving Ax = b time taken:  0.022596359252929688
epoch:  227	assembling the matrix time taken:  0.00024271011352539062
solving Ax = b time taken:  0.0228421688079834
epoch:  228	assembling the matrix time taken:  0.0002429485321044922
solving Ax = b time taken:  0.022385835647583008
epoch:  229	assembling the matrix time taken:  0.00025343894958496094
solving Ax = b time taken:  0.023119449615478516
epoch:  230	assembling the matrix time taken:  0.000244140625
solving Ax = b time taken:  0.022812604904174805
epoch:  231	assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.02330613136291504
epoch:  232	as

solving Ax = b time taken:  0.034822702407836914
epoch:  297	assembling the matrix time taken:  0.0002639293670654297
solving Ax = b time taken:  0.03585529327392578
epoch:  298	assembling the matrix time taken:  0.0002644062042236328
solving Ax = b time taken:  0.035736799240112305
epoch:  299	assembling the matrix time taken:  0.00026416778564453125
solving Ax = b time taken:  0.036302804946899414
epoch:  300	assembling the matrix time taken:  0.0002644062042236328
solving Ax = b time taken:  0.03527975082397461
epoch:  301	assembling the matrix time taken:  0.0002696514129638672
solving Ax = b time taken:  0.03633832931518555
epoch:  302	assembling the matrix time taken:  0.0002715587615966797
solving Ax = b time taken:  0.035825490951538086
epoch:  303	assembling the matrix time taken:  0.00026488304138183594
solving Ax = b time taken:  0.03610634803771973
epoch:  304	assembling the matrix time taken:  0.0002720355987548828
solving Ax = b time taken:  0.03531765937805176
epoch:  30

...

epoch:  167	assembling the matrix time taken:  0.00027871131896972656
solving Ax = b time taken:  0.015156269073486328
epoch:  168	assembling the matrix time taken:  0.00025081634521484375
solving Ax = b time taken:  0.014952659606933594
epoch:  169	assembling the matrix time taken:  0.00025200843811035156
solving Ax = b time taken:  0.015615224838256836
epoch:  170	assembling the matrix time taken:  0.0002765655517578125
solving Ax = b time taken:  0.015373468399047852
epoch:  171	assembling the matrix time taken:  0.00026297569274902344
solving Ax = b time taken:  0.015463590621948242
epoch:  172	assembling the matrix time taken:  0.0002772808074951172
solving Ax = b time taken:  0.015353679656982422
epoch:  173	assembling the matrix time taken:  0.00026035308837890625
solving Ax = b time taken:  0.01563858985900879
epoch:  174	assembling the matrix time taken:  0.0002503395080566406
solving Ax = b time taken:  0.015671968460083008
epoch:  175	assembling the matrix time taken:  0.000

epoch:  238	assembling the matrix time taken:  0.0002455711364746094
solving Ax = b time taken:  0.02365708351135254
epoch:  239	assembling the matrix time taken:  0.0002446174621582031
solving Ax = b time taken:  0.023844003677368164
epoch:  240	assembling the matrix time taken:  0.0002655982971191406
solving Ax = b time taken:  0.023626327514648438
epoch:  241	assembling the matrix time taken:  0.0002777576446533203
solving Ax = b time taken:  0.024155616760253906
epoch:  242	assembling the matrix time taken:  0.0002779960632324219
solving Ax = b time taken:  0.023998737335205078
epoch:  243	assembling the matrix time taken:  0.00024771690368652344
solving Ax = b time taken:  0.024245262145996094
epoch:  244	assembling the matrix time taken:  0.0002467632293701172
solving Ax = b time taken:  0.023955821990966797
epoch:  245	assembling the matrix time taken:  0.0002455711364746094
solving Ax = b time taken:  0.02446436882019043
epoch:  246	assembling the matrix time taken:  0.00024485

epoch:  308	assembling the matrix time taken:  0.0002703666687011719
solving Ax = b time taken:  0.03610873222351074
epoch:  309	assembling the matrix time taken:  0.00026345252990722656
solving Ax = b time taken:  0.03709053993225098
epoch:  310	assembling the matrix time taken:  0.00026297569274902344
solving Ax = b time taken:  0.03687119483947754
epoch:  311	assembling the matrix time taken:  0.0002951622009277344
solving Ax = b time taken:  0.03729557991027832
epoch:  312	assembling the matrix time taken:  0.0002903938293457031
solving Ax = b time taken:  0.036400794982910156
epoch:  313	assembling the matrix time taken:  0.00026535987854003906
solving Ax = b time taken:  0.03732442855834961
epoch:  314	assembling the matrix time taken:  0.0002911090850830078
solving Ax = b time taken:  0.037093162536621094
epoch:  315	assembling the matrix time taken:  0.0002644062042236328
solving Ax = b time taken:  0.03755950927734375
epoch:  316	assembling the matrix time taken:  0.0002887248

solving Ax = b time taken:  0.04604315757751465
epoch:  379	assembling the matrix time taken:  0.0003120899200439453
solving Ax = b time taken:  0.04639911651611328
epoch:  380	assembling the matrix time taken:  0.0002715587615966797
solving Ax = b time taken:  0.045613765716552734
epoch:  381	assembling the matrix time taken:  0.0002646446228027344
solving Ax = b time taken:  0.04666590690612793
epoch:  382	assembling the matrix time taken:  0.00028896331787109375
solving Ax = b time taken:  0.04642748832702637
epoch:  383	assembling the matrix time taken:  0.0002689361572265625
solving Ax = b time taken:  0.0469207763671875
epoch:  384	assembling the matrix time taken:  0.0002701282501220703
solving Ax = b time taken:  0.04589390754699707
epoch:  385	assembling the matrix time taken:  0.00026416778564453125
solving Ax = b time taken:  0.05624961853027344
epoch:  386	assembling the matrix time taken:  0.0002651214599609375
solving Ax = b time taken:  0.05524396896362305
epoch:  387	as

solving Ax = b time taken:  0.06785869598388672
epoch:  450	assembling the matrix time taken:  0.00029158592224121094
solving Ax = b time taken:  0.06624007225036621
epoch:  451	assembling the matrix time taken:  0.00026988983154296875
solving Ax = b time taken:  0.0662679672241211
epoch:  452	assembling the matrix time taken:  0.0002658367156982422
solving Ax = b time taken:  0.06545138359069824
epoch:  453	assembling the matrix time taken:  0.0002760887145996094
solving Ax = b time taken:  0.0667104721069336
epoch:  454	assembling the matrix time taken:  0.0002777576446533203
solving Ax = b time taken:  0.06640172004699707
epoch:  455	assembling the matrix time taken:  0.0003046989440917969
solving Ax = b time taken:  0.07228684425354004
epoch:  456	assembling the matrix time taken:  0.00027680397033691406
solving Ax = b time taken:  0.06583452224731445
epoch:  457	assembling the matrix time taken:  0.0002665519714355469
solving Ax = b time taken:  0.06739211082458496
epoch:  458	ass

## Deterministic dictionary

In [7]:
def target(x):
    return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) 

function_name = "sinpipipi" 
filename_write = "data/3DOGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 

for N_list in [[2**4,2**4,2**4],[2**4,2**4,2**3],[2**4,2**3,2**3]]: 
    save = True 
    f_write = open(filename_write, "a")
    my_model = None 
    Nx = 25   
    order = 2    
    exponent = 9  
    num_epochs = 2**exponent  
    plot_freq = 1 
    N = np.prod(N_list)
    err_QMC2, my_model = OGAL2FittingReLU3D(my_model,target,N_list,num_epochs,plot_freq, Nx, order, k =1, linear_solver = "ls")
    
    if save: 
        folder = 'data/'
        filename = folder + 'err_OGA_3D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(err_QMC2,filename) 
        folder = 'data/'
        filename = folder + 'model_OGA_3D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(my_model,filename)

    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_QMC2[i] for i in neuron_nums ]
    f_write.write('deterministic dictionary size: {}\n'.format(N))
    f_write.write("neuron num \t\t error \t\t order\n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
            f_write.write("{} \t\t {} \t\t * \n".format(neuron_nums[i],item))
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2)))
    f_write.write("\n")
    f_write.close()



using linear solver:  ls
epoch:  1	assembling the matrix time taken:  0.00024890899658203125
solving Ax = b time taken:  0.0001881122589111328
epoch:  2	assembling the matrix time taken:  0.0002319812774658203
solving Ax = b time taken:  0.0002722740173339844
epoch:  3	assembling the matrix time taken:  0.00023293495178222656
solving Ax = b time taken:  0.0002923011779785156
epoch:  4	assembling the matrix time taken:  0.00022220611572265625
solving Ax = b time taken:  0.00033593177795410156
epoch:  5	assembling the matrix time taken:  0.00024509429931640625
solving Ax = b time taken:  0.00025582313537597656
epoch:  6	assembling the matrix time taken:  0.0002665519714355469
solving Ax = b time taken:  0.0002567768096923828
epoch:  7	assembling the matrix time taken:  0.0002536773681640625
solving Ax = b time taken:  0.00029015541076660156
epoch:  8	assembling the matrix time taken:  0.000240325927734375
solving Ax = b time taken:  0.0002872943878173828
epoch:  9	assembling the matrix t

epoch:  73	assembling the matrix time taken:  0.0002868175506591797
solving Ax = b time taken:  0.0022013187408447266
epoch:  74	assembling the matrix time taken:  0.0002574920654296875
solving Ax = b time taken:  0.002285480499267578
epoch:  75	assembling the matrix time taken:  0.00024628639221191406
solving Ax = b time taken:  0.0023076534271240234
epoch:  76	assembling the matrix time taken:  0.0002472400665283203
solving Ax = b time taken:  0.002315521240234375
epoch:  77	assembling the matrix time taken:  0.00027370452880859375
solving Ax = b time taken:  0.0023069381713867188
epoch:  78	assembling the matrix time taken:  0.00024771690368652344
solving Ax = b time taken:  0.002374410629272461
epoch:  79	assembling the matrix time taken:  0.0002586841583251953
solving Ax = b time taken:  0.0024101734161376953
epoch:  80	assembling the matrix time taken:  0.00024962425231933594
solving Ax = b time taken:  0.0023965835571289062
epoch:  81	assembling the matrix time taken:  0.0002851

epoch:  145	assembling the matrix time taken:  0.00028133392333984375
solving Ax = b time taken:  0.005753040313720703
epoch:  146	assembling the matrix time taken:  0.0002799034118652344
solving Ax = b time taken:  0.005568981170654297
epoch:  147	assembling the matrix time taken:  0.0002734661102294922
solving Ax = b time taken:  0.005719661712646484
epoch:  148	assembling the matrix time taken:  0.0002715587615966797
solving Ax = b time taken:  0.0055999755859375
epoch:  149	assembling the matrix time taken:  0.0002970695495605469
solving Ax = b time taken:  0.005767822265625
epoch:  150	assembling the matrix time taken:  0.0002892017364501953
solving Ax = b time taken:  0.00577235221862793
epoch:  151	assembling the matrix time taken:  0.0002701282501220703
solving Ax = b time taken:  0.005919218063354492
epoch:  152	assembling the matrix time taken:  0.0002684593200683594
solving Ax = b time taken:  0.005715131759643555
epoch:  153	assembling the matrix time taken:  0.000271558761

epoch:  216	assembling the matrix time taken:  0.0002677440643310547
solving Ax = b time taken:  0.009451627731323242
epoch:  217	assembling the matrix time taken:  0.0002803802490234375
solving Ax = b time taken:  0.009673595428466797
epoch:  218	assembling the matrix time taken:  0.000308990478515625
solving Ax = b time taken:  0.009772777557373047
epoch:  219	assembling the matrix time taken:  0.0002815723419189453
solving Ax = b time taken:  0.009917736053466797
epoch:  220	assembling the matrix time taken:  0.00027823448181152344
solving Ax = b time taken:  0.009705543518066406
epoch:  221	assembling the matrix time taken:  0.00026679039001464844
solving Ax = b time taken:  0.009942770004272461
epoch:  222	assembling the matrix time taken:  0.00026726722717285156
solving Ax = b time taken:  0.009933710098266602
epoch:  223	assembling the matrix time taken:  0.0002758502960205078
solving Ax = b time taken:  0.010040521621704102
epoch:  224	assembling the matrix time taken:  0.00026

solving Ax = b time taken:  0.015682458877563477
epoch:  287	assembling the matrix time taken:  0.00027370452880859375
solving Ax = b time taken:  0.015540122985839844
epoch:  288	assembling the matrix time taken:  0.00028514862060546875
solving Ax = b time taken:  0.015286922454833984
epoch:  289	assembling the matrix time taken:  0.0002875328063964844
solving Ax = b time taken:  0.015834331512451172
epoch:  290	assembling the matrix time taken:  0.00027370452880859375
solving Ax = b time taken:  0.015662431716918945
epoch:  291	assembling the matrix time taken:  0.0002758502960205078
solving Ax = b time taken:  0.01588726043701172
epoch:  292	assembling the matrix time taken:  0.00030517578125
solving Ax = b time taken:  0.015626192092895508
epoch:  293	assembling the matrix time taken:  0.00027179718017578125
solving Ax = b time taken:  0.015552997589111328
epoch:  294	assembling the matrix time taken:  0.00027251243591308594
solving Ax = b time taken:  0.015949726104736328
epoch:  

...

solving Ax = b time taken:  0.008365869522094727
epoch:  200	assembling the matrix time taken:  0.0002608299255371094
solving Ax = b time taken:  0.008329391479492188
epoch:  201	assembling the matrix time taken:  0.0002799034118652344
solving Ax = b time taken:  0.008441448211669922
epoch:  202	assembling the matrix time taken:  0.00026488304138183594
solving Ax = b time taken:  0.008644819259643555
epoch:  203	assembling the matrix time taken:  0.00029206275939941406
solving Ax = b time taken:  0.008606433868408203
epoch:  204	assembling the matrix time taken:  0.0002689361572265625
solving Ax = b time taken:  0.008578777313232422
epoch:  205	assembling the matrix time taken:  0.0002601146697998047
solving Ax = b time taken:  0.00876617431640625
epoch:  206	assembling the matrix time taken:  0.0002646446228027344
solving Ax = b time taken:  0.008769035339355469
epoch:  207	assembling the matrix time taken:  0.0002636909484863281
solving Ax = b time taken:  0.008814573287963867
epoch:

solving Ax = b time taken:  0.012739419937133789
epoch:  273	assembling the matrix time taken:  0.00026988983154296875
solving Ax = b time taken:  0.01334834098815918
epoch:  274	assembling the matrix time taken:  0.0003058910369873047
solving Ax = b time taken:  0.013272285461425781
epoch:  275	assembling the matrix time taken:  0.0002734661102294922
solving Ax = b time taken:  0.013334035873413086
epoch:  276	assembling the matrix time taken:  0.0002777576446533203
solving Ax = b time taken:  0.013030529022216797
epoch:  277	assembling the matrix time taken:  0.00028014183044433594
solving Ax = b time taken:  0.013428211212158203
epoch:  278	assembling the matrix time taken:  0.0002741813659667969
solving Ax = b time taken:  0.013609170913696289
epoch:  279	assembling the matrix time taken:  0.0002727508544921875
solving Ax = b time taken:  0.013524055480957031
epoch:  280	assembling the matrix time taken:  0.00029730796813964844
solving Ax = b time taken:  0.013162612915039062
epoch

epoch:  344	assembling the matrix time taken:  0.0002739429473876953
solving Ax = b time taken:  0.016511201858520508
epoch:  345	assembling the matrix time taken:  0.0002751350402832031
solving Ax = b time taken:  0.017265796661376953
epoch:  346	assembling the matrix time taken:  0.00027179718017578125
solving Ax = b time taken:  0.017107248306274414
epoch:  347	assembling the matrix time taken:  0.00027370452880859375
solving Ax = b time taken:  0.017384052276611328
epoch:  348	assembling the matrix time taken:  0.0002720355987548828
solving Ax = b time taken:  0.017084360122680664
epoch:  349	assembling the matrix time taken:  0.00028014183044433594
solving Ax = b time taken:  0.017607688903808594
epoch:  350	assembling the matrix time taken:  0.00029850006103515625
solving Ax = b time taken:  0.017377614974975586
epoch:  351	assembling the matrix time taken:  0.0002758502960205078
solving Ax = b time taken:  0.01710653305053711
epoch:  352	assembling the matrix time taken:  0.0002

solving Ax = b time taken:  0.022873640060424805
epoch:  416	assembling the matrix time taken:  0.00027179718017578125
solving Ax = b time taken:  0.022182226181030273
epoch:  417	assembling the matrix time taken:  0.00026988983154296875
solving Ax = b time taken:  0.023302793502807617
epoch:  418	assembling the matrix time taken:  0.00027370452880859375
solving Ax = b time taken:  0.0226287841796875
epoch:  419	assembling the matrix time taken:  0.00027251243591308594
solving Ax = b time taken:  0.023068904876708984
epoch:  420	assembling the matrix time taken:  0.0002758502960205078
solving Ax = b time taken:  0.022272825241088867
epoch:  421	assembling the matrix time taken:  0.0002734661102294922
solving Ax = b time taken:  0.023035287857055664
epoch:  422	assembling the matrix time taken:  0.0002713203430175781
solving Ax = b time taken:  0.022965431213378906
epoch:  423	assembling the matrix time taken:  0.00027441978454589844
solving Ax = b time taken:  0.02314591407775879
epoch

solving Ax = b time taken:  0.027773141860961914
epoch:  486	assembling the matrix time taken:  0.000308990478515625
solving Ax = b time taken:  0.027519702911376953
epoch:  487	assembling the matrix time taken:  0.00027489662170410156
solving Ax = b time taken:  0.027978897094726562
epoch:  488	assembling the matrix time taken:  0.0002810955047607422
solving Ax = b time taken:  0.027101993560791016
epoch:  489	assembling the matrix time taken:  0.0002570152282714844
solving Ax = b time taken:  0.028020858764648438
epoch:  490	assembling the matrix time taken:  0.0002567768096923828
solving Ax = b time taken:  0.028105497360229492
epoch:  491	assembling the matrix time taken:  0.00026106834411621094
solving Ax = b time taken:  0.028179168701171875
epoch:  492	assembling the matrix time taken:  0.0002810955047607422
solving Ax = b time taken:  0.027447938919067383
epoch:  493	assembling the matrix time taken:  0.0002570152282714844
solving Ax = b time taken:  0.0281219482421875
epoch:  